In [4]:
import itertools
from collections import defaultdict
import random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
import matplotlib.pyplot as plt

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
test_path = '/content/drive/My Drive/circuits/c17.test'

In [7]:
# ------------------ Reproducibility ------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# ------------------ Expand Don't Care ------------------
def expand_dont_care(pattern):
    chars = [(c if c in '01' else ['0', '1']) for c in pattern]
    chars = [(c if isinstance(c, list) else [c]) for c in chars]
    return [''.join(bits) for bits in itertools.product(*chars)]

# ------------------ Parse .test File ------------------
def parse_isc_file(filepath):
    combo_to_faults = defaultdict(set)
    fault_to_combos = defaultdict(list)
    current_fault = None

    with open(filepath, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith("*"):
                continue
            if '/' in line:
                parts = line.split("/")
                fault = parts[0].strip().replace("->", "_") + "/" + parts[1].strip()
                current_fault = fault
            elif ':' in line and current_fault:
                parts = line.split(":")[1].strip().split()
                input_pattern = parts[0]
                output_pattern = parts[1] if len(parts) > 1 else ""
                input_expanded = expand_dont_care(input_pattern)
                output_expanded = expand_dont_care(output_pattern)
                for xi in input_expanded:
                    for yo in output_expanded:
                        combined = xi + yo
                        combo_to_faults[combined].add(current_fault)
                        fault_to_combos[current_fault].append(combined)
    return combo_to_faults, fault_to_combos

# ------------------ Dataset Builder ------------------
def build_multilabel_dataset(combo_to_faults):
    all_faults = sorted({f for faults in combo_to_faults.values() for f in faults})
    fault_index = {fault: idx for idx, fault in enumerate(all_faults)}
    dataset = []
    for combo, faults in combo_to_faults.items():
        label_vector = [0] * len(all_faults)
        for fault in faults:
            label_vector[fault_index[fault]] = 1
        features = [int(bit) for bit in combo]
        dataset.append((features, label_vector))
    return dataset, fault_index

In [8]:
# ------------------ Load and Prepare ------------------
combo_to_faults, fault_to_combos = parse_isc_file(test_path)
dataset, fault_index = build_multilabel_dataset(combo_to_faults)

X = np.array([x for x, _ in dataset])
y = np.array([y for _, y in dataset])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [9]:
# ------------------ LASSO Classifier ------------------
lasso = OneVsRestClassifier(
    LogisticRegression(penalty='l1', solver='saga', max_iter=1000, random_state=SEED)
)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_val)

# ------------------ Evaluation ------------------
f1 = f1_score(y_val, y_pred, average='macro', zero_division=0)
overall_acc = (y_val == y_pred).sum() / y_val.size

# ------------------ Fault-Type Accuracy ------------------
stuck_at_0_indices = [idx for fault, idx in fault_index.items() if fault.endswith('/0')]
stuck_at_1_indices = [idx for fault, idx in fault_index.items() if fault.endswith('/1')]

def type_accuracy(y_true, y_pred, indices, fault_type):
    y_true_type = y_true[:, indices]
    y_pred_type = y_pred[:, indices]
    correct = (y_true_type == y_pred_type).sum()
    total = y_true_type.size
    acc = correct / total if total > 0 else 0
    print(f"{fault_type} Accuracy: {acc:.4f} ({correct}/{total})")
    return acc

# ------------------ Print Final Metrics ------------------
print(f"\nValidation F1 Score: {f1:.4f}")
print(f"Final Overall Accuracy: {overall_acc:.4f} ({(overall_acc * y_val.size):.0f}/{y_val.size})\n")
print("--- Fault-Type Accuracy ---")
type_accuracy(y_val, y_pred, stuck_at_0_indices, "Stuck-at-0")
type_accuracy(y_val, y_pred, stuck_at_1_indices, "Stuck-at-1")


Validation F1 Score: 0.6449
Final Overall Accuracy: 0.9606 (830/864)

--- Fault-Type Accuracy ---
Stuck-at-0 Accuracy: 0.9259 (200/216)
Stuck-at-1 Accuracy: 0.9722 (630/648)


np.float64(0.9722222222222222)